# Transfer learning

In [15]:
import time
import torch
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
import torchvision
from torchvision import transforms

In [16]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [18]:
train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=64, shuffle=False)

Files already downloaded and verified


In [19]:
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

Files already downloaded and verified


In [20]:
model = torchvision.models.alexnet(pretrained=True)

In [21]:
model.classifier[6].in_features

4096

In [22]:
for param in model.parameters():
    param.requires_grad = False

In [23]:
model.classifier[6] = nn.Linear(4096, 10)

In [24]:
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [25]:
sum([p.numel() for p in model.parameters() if p.requires_grad])

40970

In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.classifier[6].parameters(), lr=0.001)

In [27]:
model = model.to(DEVICE)

In [28]:
def training_loop(n_epochs, optimizer, model, criterion, train_loader):
    start = time.time()
    for epoch in range(n_epochs):
        losses = []
        
        for imgs, labels in train_loader:
            imgs = imgs.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            
            losses.append(loss.item())
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        print(f"Epoch: {epoch}; loss: {sum(losses)/len(losses)}")
    end = time.time()
    print(f"Czas: {round(end - start)} s")

In [15]:
training_loop(n_epochs=10, optimizer=optimizer, model=model, 
              criterion=criterion, train_loader=train_loader)

Epoch: 0; loss: 1.0984905145662216
Epoch: 1; loss: 0.8165860701247555
Epoch: 2; loss: 0.761353334807374
Epoch: 3; loss: 0.7271444379063823
Epoch: 4; loss: 0.7061305258356397
Epoch: 5; loss: 0.6895467759398244
Epoch: 6; loss: 0.6797113422009037
Epoch: 7; loss: 0.6714341939638948
Epoch: 8; loss: 0.6641468965946256
Epoch: 9; loss: 0.6522095074205447
Czas: 1003 s


In [16]:
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [17]:
def get_accuracy(model, data_loader):
    correct = 0
    total = 0

    with torch.no_grad():
        for xs, ys in data_loader:
            xs = xs.to(DEVICE)
            ys = ys.to(DEVICE)
            scores = model(xs)
            _, predictions = torch.max(scores, dim=1)
            correct += (predictions == ys).sum()
            total += ys.shape[0]

        acc = float(correct) / float(total) * 100
        print(f"accuracy: {acc:.2f}")

In [18]:
get_accuracy(model, train_loader)

accuracy: 81.01


In [19]:
get_accuracy(model, test_loader)

accuracy: 80.01
